In [142]:
import numpy as np
import pandas as pd 
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
import missingno as mno
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
import time
import torch
from torch.optim.lr_scheduler import *
from sklearn.linear_model import LinearRegression as LR
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from torchsummary import summary
from torch.autograd import Variable
from sklearn.metrics import *
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA
import time

In [143]:
data=pd.read_csv('observations_data.csv')
data=data.dropna()
data = data.rename(index=str,columns= {'SID Brest: Limni_3_mH2O[mH20]': "SID Brest: Limni_3_mH2O[mH20]"}) 
data['Time'] = pd.to_datetime(data['Time'])

In [144]:
# checking nan values for data frame
def check_nan_values(dataframe):
    for column in dataframe.columns:
        check=dataframe[column].isnull().values.any()
        if check==True:
            print(column, 'missing data: ',dataframe[column].isnull().sum())
    else : print('dataframe on point')

##time prep is computed for standard scaled time and for seasonal term whether it's used or not 
def time_prep(dataframe,column='Time'):
    window_observation=dataframe[column][len(dataframe)-1]-dataframe[column][0]
    dataframe['scaled_time']=dataframe[column].apply(lambda T :((T-dataframe[column][0]).total_seconds())/window_observation.total_seconds())
    ## preaparing seasonal time for linear combination of sinusoidal functions
    dataframe['seasonal_time']=dataframe[column].apply(lambda T :(2*np.pi*(T-dataframe[column][0]).total_seconds())/365*24*60*60)
    return dataframe


##hydrostatic load
def hydrostatic_load(dataframe,wl_column,regression_order=4):
    hydrostaic_columns=[]
    max_observation=np.max(dataframe[wl_column])
    min_observation=np.min(dataframe[wl_column])
    scaled_observation=(max_observation-dataframe[wl_column])/(max_observation-min_observation)
    dataframe['scaled_observation']=scaled_observation
    for k in range(regression_order+1):
        order='HL_'+str(k)
        dataframe[order]=scaled_observation**k
        hydrostaic_columns.append(order)
    return dataframe,hydrostaic_columns
#seasonal term

def seasonal_term(dataframe):
    #scaled_time=dataframe['seasonal_time']
    season_columns=['sin','cos','sin2','sincos']
    dataframe[season_columns[0]]=np.sin(dataframe['seasonal_time'])
    dataframe[season_columns[1]]=np.cos(dataframe['seasonal_time'])
    dataframe[season_columns[2]]=np.sin(dataframe['seasonal_time'])**2
    dataframe[season_columns[3]]=np.sin(dataframe['seasonal_time'])*np.cos(dataframe['seasonal_time'])
    return dataframe,season_columns

#long term time influence


def Time_influence(dataframe):
    Time_columns=['T1','T2','T3']
    dataframe[Time_columns[0]]=dataframe['scaled_time']
    dataframe[Time_columns[1]]=np.exp(dataframe['scaled_time'])
    dataframe[Time_columns[2]]=np.exp((-1)*dataframe['scaled_time'])
    return dataframe , Time_columns

"""
Following "Hybrid GA/SIMPLS as alternative regression model in dam deformation analysis" paper 
the temperature componenent is computed conditionnaly on its observation existance. if it doesn't 
we compute a sum of sin and cos based on seasonal time
"""

def temperature_component(data,temp_column):
    if temp_column is not None :
        scale=np.max(data[temp_column])-np.min(data[temp_column])
        data['temperature']=data[temp_column]-np.mean(data[temp_column])
        data['temperature']/=scale
        return data,['temperature']
    else:
        Temp_columns=[]
        for k in range(1,6):
            data['sin_temp_'+str(k)]=np.sin(k*data['seasonal_time'])
            data['cos_temp_'+str(k)]=np.cos(k*data['seasonal_time'])
            Temp_columns.append('sin_temp_'+str(k),'cos_temp_'+str(k))
    return data ,Temp_columns

def pression_component(data,pression_column):
    scale=np.max(data[pression_column])-np.min(data[pression_column])
    data['pression']=data[pression_column]-np.mean(data[pression_column])
    data['pression']/=scale
    return data , ['pression']

def inclinaison_component(data,inclinaison_column):
    scale=np.max(data[inclinaison_column])-np.min(data[inclinaison_column])
    data['inclinaison']=data[inclinaison_column]-np.mean(data[inclinaison_column])
    data['inclinaison']/=scale
    return data,['inclinaison']
def component(data,component_column):
    model_component=component_column+'model_component'
    scale=np.max(data[inclinaison_column])-np.min(data[inclinaison_column])
    data[model_component]=data[inclinaison_column]-np.mean(data[inclinaison_column])
    data[model_component]/=scale
    return data,[model_component]

In [145]:
wl_column='SID Brest: Limni_3_mH2O[mH20]'
temp_column='SID Brest: PTemp[N/A]'
pression_column='SID Brest: SONDE_04_kPa[kPa]'
inclinaison_column= 'SID Brest: INC_IPI03_°[°]'

In [146]:
def prepare_prediction(dataframe,model):
    df=dataframe.copy()
    df=time_prep(df)
    kept_columns=[]
    if 'H' in model :
        df,columns=hydrostatic_load(df,wl_column,regression_order=4)
        kept_columns+=columns
    if 'S' in model:
        df,columns=seasonal_term(df)
        kept_columns+=columns
    if 'T' in model :
        df,columns=Time_influence(df)
        kept_columns+=columns
    if 'temp' in model :
        df,columns=temperature_component(df,temp_column)
        kept_columns+=columns
    if 'P' in model :
        df,columns=pression_component(df,pression_column)
        kept_columns+=columns
    if 'I' in model :
        df,columns=inclinaison_component(df,inclinaison_column)
        kept_columns+=columns
        
   # kept_columns=['Time',wl_column,'sin','cos','sin2','sincos','T1','T2','T3','HL_0','HL_1','HL_2','HL_3','HL_4']
    return df[kept_columns]


In [147]:
model='HSTtemp'
target_column= data.columns[3]
matrix=np.array(prepare_prediction(data,model)).T
target=np.array(data[target_column])
print(target.shape,matrix.shape)

(59882,) (13, 59882)


In [68]:
def linear_regression(data,target):
    #monitored_=data[pd.notnull(data[target_column])]
    #monitored_=monitored_[pd.notnull(monitored_[wl_column])]
    start=time.time()
    #dataframe=prepare_prediction(data,model)   
    #print('computed variables in :' ,time.time()-start)
    #check_nan_values(data)
    regr =LinearRegression()
    #regr.fit(dataframe,monitored_[target_column])
    regr.fit(data,target)
    #print( 'regr score for '+target_column+'is:' , regr.score(data,target))
    #dataframe.insert(0,'corrected observation',monitored_[target_column] -np.array(regr.predict(dataframe)))
    #dataframe.insert(len(dataframe.columns),target_column,monitored_[target_column])
    return regr

In [70]:
def regression_kfold(matrix,target,n_splits):
    cv_outer = KFold(n_splits, shuffle=True, random_state=0)
    # enumerate splits
    outer_results = list()
    X=matrix.T
    y=target
    best_score=np.NINF
    best_regr=0,0
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]
        #cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
        #__,weights,biais=optimizer_variation(X_train.T,  y_train,  'Adam',  lr=0.1,  num_epoch=2500)
        regr=linear_regression(X_train,y_train)
        y_hat=regr.predict(X_test)
        score=r2_score(y_hat,y_test)
        """
        y_hat=weights.mm(torch.Tensor(X_test.T)) + biais
        y_hat=y_hat.squeeze()
        score=r2_score(y_test,y_hat.detach().numpy())"""

        if score>best_score :
            best_score=score
            best_regr=regr
    print(f"final score : {score :.3f}")
    return best_regr

In [71]:
def standard_regression(data,target,use_cross_val):
    if use_cross_val:
        regr=regression_kfold(matrix,target,n_splits=3)
        y_hat=regr.predict(data)
        ##print( 'regr score for '+target_column+'is:' , regr.score(dataframe,monitored_[target_column]))
        #dataframe.insert(0,'corrected observation',monitored_[target_column] -np.array(regr.predict(dataframe)))
        #dataframe.insert(len(dataframe.columns),target_column,monitored_[target_column])
    else:
        regr=linear_regression(data,target)
        y_hat=regr.predict(data)
    return y_hat

In [76]:
standard_regression(matrix.T,target,use_cross_val=False)

array([-0.07879281, -0.07194117, -0.06988126, ..., -2.0463267 ,
       -2.05179639, -2.12755343])

In [85]:

# Main optimization loop
def optimizer_variation(matrix,  target,  optimizer,  lr=0.01,  num_epoch=2500):
    A = torch.randn((1, matrix.shape[0]), requires_grad=True)
    b = torch.randn(1, requires_grad=True)
    if optimizer=='SGD': 
        opt = torch.optim.SGD([A, b], lr=lr)
    elif optimizer=='Adam' : 
        opt=torch.optim.Adam([A, b], lr=lr)
    elif optimizer=='AdamW':
        opt=torch.optim.AdamW([A, b], lr=lr)
    elif optimizer=='Adamax':
        opt=torch.optim.AdamW([A, b], lr=lr)
    elif optimizer=='ASGD':
        opt=torch.optim.ASGD([A, b], lr=lr)
    elif optimizer=='Adagrad':
        opt=torch.optim.Adagrad([A, b], lr=lr)
    ##setting up a learning rate scheduler
    criterion=torch.nn.MSELoss()
    scheduler = ExponentialLR(opt, gamma=0.1)
    ##Multi Linear type model 
    def NN(x_input):
        return A.mm(x_input) + b
    best_score=np.NINF
    best_loss=np.inf
    best_pred=0
    ##Trainning loop 
    for t in range(0,num_epoch):
        opt.zero_grad()
        y_predicted = NN(torch.Tensor(matrix))
        current_loss = criterion(y_predicted, torch.Tensor(target))
        current_loss.backward()
        opt.step()
        pred=y_predicted.detach().numpy().squeeze()
        score=r2_score(target,pred)
        ##keeping best model
        if score>best_score : 
            best_score=score
            best_pred=target-pred
            best_loss=current_loss
            weights=A
            biais=b
    #print(f"t = {t}, loss: {best_loss:.2F}, score : {score :.3f}")
    prediction=NN(torch.Tensor(matrix))
    return prediction,weights,biais


In [206]:

# create dataset
# configure the cross-validation procedure
def k_fold_cross(matrix,target,optimizer):
    cv_outer = KFold(n_splits=6, shuffle=True, random_state=0)
    # enumerate splits
    outer_results = list()
    X=matrix.T
    y=target
    best_score=np.NINF
    best_weights,best_biais=0,0
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]
        #cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)

        __,weights,biais=optimizer_variation(X_train.T,  y_train,  optimizer,  lr=0.1,  num_epoch=1000)
        y_hat=weights.mm(torch.Tensor(X_test.T)) + biais
        y_hat=y_hat.squeeze()
        score=r2_score(y_test,y_hat.detach().numpy())

        if score>best_score :
            best_score=score
            best_weights=weights
            best_biais=biais
    print(f"final score : {score :.3f}")
    final_pred=weights.mm(torch.Tensor(matrix)) + biais
    return final_pred


In [220]:
def calculate_w(l, phi, t, N_rows, M_cols):
    LI = np.eye(M_cols) * l
    innerPrdt = (LI + np.matmul(np.transpose(phi),phi))
    w = np.matmul(inv(innerPrdt),np.matmul(np.transpose(phi),t))
    return w
def bayesian_model_selection(train, trainR, test, testR):
    start_time = time.time()
    alpha = 2.34
    beta = 3.22
    prev_alpha = 0
    prev_beta = 0
    N_rows_train = np.shape(train)[0]
    M_features_train = np.shape(train)[1]
    N_rows_test = np.shape(test)[0]
    i = 0
    while abs(prev_alpha-alpha) > 0.0001 and abs(prev_beta-beta) > 0.0001:
        prev_alpha = alpha
        prev_beta = beta
        eigen_phiT_phi = LA.eigvals(np.matmul(np.transpose(train),train))
        Sn_inv = alpha * np.eye(M_features_train) + beta * np.matmul(np.transpose(train),train)
        Mn = beta * np.matmul(inv(Sn_inv),np.matmul(np.transpose(train),trainR))
        lamda = beta * eigen_phiT_phi
        alpha_lamda = LA.eigvals(Sn_inv)
        gamma = sum(lamda/alpha_lamda)
        alpha = gamma / np.matmul(np.transpose(Mn),Mn)
        beta = 1/(sum((trainR - np.matmul(train,Mn)) ** 2) / (N_rows_train-gamma))
        i+=1

    w = calculate_w(alpha/beta,train,trainR,N_rows_train,M_features_train)
    mse = (sum((np.matmul(test,w) - testR) ** 2) / N_rows_test)
    print("computed bayesian model in ", time.time()-start_time)
    return mse,w

def bayesian_selection(matrix,target,n_splits=2):
    X=matrix.T
    y=target
    
    if n_splits==1 :
        X_train, y_train, X_test, y_test=train_test_split(X,y,test_size = 0.25,shuffle=False)
        return bayesian_model_selection(X_train, y_train, X_test, y_test)
    else:
        cv_outer = KFold(n_splits=n_splits, shuffle=True, random_state=0)
            # enumerate splits

        best_score=np.NINF
        best_weights,best_biais=0,0
        best_score=np.NINF
        best_weight=0
        for train_ix, test_ix in cv_outer.split(X):
            X_train, X_test = X[train_ix, :], X[test_ix, :]
            y_train, y_test = y[train_ix], y[test_ix]
            mse,weight=bayesian_model_selection(X_train, y_train, X_test, y_test)
            if mse>best_score :
                best_score=mse
                best_weight=weight
        best_weight=best_weight.reshape(best_weight.shape[0])
    return np.dot(X,best_weight)

In [221]:
bayesian_selection(matrix,target,n_splits=2).shape

computed bayesian model in  0.025943756103515625
computed bayesian model in  0.04488015174865723


(59882,)

In [222]:
model='HSTtemp'
target_column= data.columns[3]

wl_column='SID Brest: Limni_3_mH2O[mH20]'
temp_column='SID Brest: PTemp[N/A]'
pression_column='SID Brest: SONDE_04_kPa[kPa]'
inclinaison_column= 'SID Brest: INC_IPI03_°[°]'

optimizer='SGD'

methods_list=['std_regression','std_regression_cross','use_NN_optimizer','use_NN_optimizer_cross'
             'use_bayesian_approch']

def deploy(model,data,target_column,method):
    matrix=np.array(prepare_prediction(data,model)).T
    target=np.array(data[target_column])
    dataframe=data.copy()
    if method==methods_list[0]:
        corrected_obs=standard_regression(matrix.T,target,use_cross_val=False)
        dataframe.insert(0,'corrected observation',data[target_column] -np.array(corrected_obs))
    elif  method==methods_list[1]:
        corrected_obs=standard_regression(matrix.T,target,use_cross_val=True)
        dataframe.insert(0,'corrected observation',data[target_column] -np.array(corrected_obs))
    elif  method==methods_list[2]:
        corrected_obs,__,__=optimizer_variation(matrix,  target,  optimizer='SGD',  lr=0.1,  num_epoch=2500)
        dataframe.insert(0,'corrected observation',data[target_column] -corrected_obs.squeeze().detach().numpy())
    elif method==methods_list[3]:
        corrected_obs=k_fold_cross(matrix,target,optimizer)
        dataframe.insert(0,'corrected observation',data[target_column] -corrected_obs.squeeze().detach().numpy())
    elif method==methods_list[4]:
        corrected_obs=bayesian_selection(matrix,target,n_splits=2)
        dataframe.insert(0,'corrected observation',data[target_column] -np.array(corrected_obs))

    return dataframe
        
    
    
    
    
    
    
    
    
    
    
    
    
    

In [223]:
model='HSTtemp'

data=pd.read_csv('observations_data.csv')
data=data.dropna()
data = data.rename(index=str,columns= {'SID Brest: Limni_3_mH2O[mH20]': "SID Brest: Limni_3_mH2O[mH20]"}) 
data['Time'] = pd.to_datetime(data['Time'])


target_column= data.columns[3]
wl_column='SID Brest: Limni_3_mH2O[mH20]'
temp_column='SID Brest: PTemp[N/A]'
pression_column='SID Brest: SONDE_04_kPa[kPa]'
inclinaison_column= 'SID Brest: INC_IPI03_°[°]'

optimizer='SGD'


methods_list=['std_regression','std_regression_cross','use_NN_optimizer','use_NN_optimizer_cross',
             'use_bayesian_approch']

In [224]:
 deploy(model,data,target_column,methods_list[4])

computed bayesian model in  0.022643327713012695
computed bayesian model in  0.02618694305419922


,corrected observation,Unnamed: 0,Time,SID Brest: FIS_05A_mm[mm],SID Brest: FIS_06A_mm[mm],SID Brest: FIS_07A_mm[mm],SID Brest: FIS_07B_mm[mm],SID Brest: FIS_07C_mm[mm],SID Brest: FIS_08A_mm[mm],SID Brest: FIS_08B_mm[mm],...,SID Brest: INC_IPI02_°[°],SID Brest: INC_IPI03_°[°],SID Brest: SONDE_02_kPa[kPa],SID Brest: SONDE_03_kPa[kPa],SID Brest: SONDE_04_kPa[kPa],SID Brest: SONDE_05_kPa[N/A],SID Brest: SONDE_06_kPa[N/A],SID Brest: SONDE_07_kPa[kPa],SID Brest: SONDE_08_kPa[kPa],SID Brest: SONDE_09_kPa[kPa]
41113,-0.233607,41113,2014-12-01 16:30:00,0.022518,-0.308988,0.686100,0.075958,0.218682,-7.392567,0.081965,...,0.047596,-0.020033,263.780205,813.443929,38.576279,662.647531,690.706811,680.976278,713.513551,757.113941
41114,-0.240767,41114,2014-12-01 16:40:00,0.022800,-0.308548,0.699113,0.075322,0.228718,-7.392681,0.078765,...,0.047644,-0.020043,262.753342,813.521063,10.168108,704.486243,694.866924,724.056018,717.953635,757.050743
41115,-0.243405,41115,2014-12-01 16:50:00,0.022913,-0.308676,0.691784,0.075041,0.208492,-7.392409,0.074061,...,0.047717,-0.020013,261.737879,10.705809,17.533413,-22.993509,694.613965,1.552368,717.798099,-3.773278
41116,-0.249516,41116,2014-12-01 17:00:00,0.023044,-0.308658,0.691265,0.074891,0.217491,-7.392273,0.074381,...,0.047669,-0.019983,260.737164,843.053776,19.097680,731.430192,695.062400,-9.245302,717.720382,725.995055
41117,-0.245154,41117,2014-12-01 17:10:00,0.023776,-0.308585,0.691524,0.074947,0.224453,-7.392409,0.070189,...,0.047705,-0.020013,259.759520,38.933570,709.394051,666.769332,691.397409,685.568127,36.873515,18.068219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178082,1.563431,178082,2018-07-10 08:50:00,-0.029231,-0.594282,-0.801347,0.105930,-0.191293,0.512000,0.043254,...,0.164512,0.009986,273.584586,176.077237,170.802740,175.330676,172.319984,163.793045,173.539383,196.214133
178083,1.559141,178083,2018-07-10 09:00:00,-0.030488,-0.596287,-0.799834,0.105631,-0.181018,0.509974,0.063566,...,0.164585,0.009926,273.961763,176.235189,170.747348,175.224469,172.380577,163.700822,173.448022,196.229728
178084,1.549130,178084,2018-07-10 09:10:00,-0.030919,-0.598601,-0.806867,0.105649,-0.177437,0.507846,0.096337,...,0.164585,0.009936,274.520265,176.617972,170.988019,175.464923,172.723970,163.946948,173.683456,195.542907
178085,1.548608,178085,2018-07-10 09:20:00,-0.035419,-0.601626,-0.810649,0.106099,-0.186281,0.505332,0.122689,...,0.164682,0.009866,275.136288,177.043926,171.247770,175.724878,173.100056,164.211553,173.934055,195.657066
